<!-- HTML file automatically generated from DocOnce source (https://github.com/doconce/doconce/)
doconce format html week8.do.txt --no_mako -->
<!-- dom:TITLE: March 13-17,2023: Quantum Computing, Quantum Machine Learning and Quantum Information Theories -->

# March 13-17,2023: Quantum Computing, Quantum Machine Learning and Quantum Information Theories
**Morten Hjorth-Jensen**, Department of Physics, University of Oslo and Department of Physics and Astronomy and National Superconducting Cyclotron Laboratory, Michigan State University

Date: **Mar 20, 2023**

Copyright 1999-2023, Morten Hjorth-Jensen. Released under CC Attribution-NonCommercial 4.0 license

## Solving quantum mechanical problems

1. Simple Hamiltonian, the Lipkin model

2. Introducing the Variational Quantum Eigensolver (VQE)

* [Video of lecture](https://youtu.be/ffIwxiUJGT4)

## Simple Hamiltonian, the Lipkin model

We will study a schematic model (the Lipkin model, see Nuclear
Physics **62** (1965) 188), for the interaction among  $2$ and more 
fermions that can occupy two different energy levels. For four fermions, the case we consider first here, each levels has degeneration $d=4$. The two levels have quantum numbers $\sigma=\pm 1$,
with the upper level having  $\sigma=+1$ and energy
$\varepsilon_{1}=
\varepsilon/2$. The lower level  has $\sigma=-1$ and energy
$\varepsilon_{2}=-\varepsilon/2$. 
In addition, the substates  of each level are characterized  
by the quantum numbers $p=1,2,3,4$.

We define the single-particle states (for the four fermion case which we will work on here)

$$
\vert u_{\sigma =-1,p}\rangle=a_{-p}^{\dagger}\vert 0\rangle
\hspace{1cm}
\vert u_{\sigma =1,p}\rangle=a_{+p}^{\dagger}\vert 0\rangle.
$$

The single-particle states span an orthonormal basis.
The Hamiltonian of the system is given by

$$
\begin{array}{ll}
\hat{H}=&\hat{H}_{0}+\hat{H}_{1}+\hat{H}_{2}\\
&\\
\hat{H}_{0}=&\frac{1}{2}\varepsilon\sum_{\sigma ,p}\sigma
a_{\sigma,p}^{\dagger}a_{\sigma ,p}\\
&\\
\hat{H}_{1}=&\frac{1}{2}V\sum_{\sigma ,p,p'}
a_{\sigma,p}^{\dagger}a_{\sigma ,p'}^{\dagger}
a_{-\sigma ,p'}a_{-\sigma ,p}\\
&\\
\hat{H}_{2}=&\frac{1}{2}W\sum_{\sigma ,p,p'}
a_{\sigma,p}^{\dagger}a_{-\sigma ,p'}^{\dagger}
a_{\sigma ,p'}a_{-\sigma ,p}\\
&\\
\end{array}
$$

where $V$ and $W$ are constants. The operator 
$H_{1}$ can move pairs of fermions
while $H_{2}$ is a spin-exchange term. The latter
moves a pair of fermions from a state $(p\sigma ,p' -\sigma)$ to a state
$(p-\sigma ,p'\sigma)$.

We are going to rewrite the above Hamiltonian in terms of so-called  quasispin operators

$$
\begin{array}{ll}
\hat{J}_{+}=&\sum_{p}
a_{p+}^{\dagger}a_{p-}\\
&\\
\hat{J}_{-}=&\sum_{p}
a_{p-}^{\dagger}a_{p+}\\
&\\
\hat{J}_{z}=&\frac{1}{2}\sum_{p\sigma}\sigma
a_{p\sigma}^{\dagger}a_{p\sigma}\\
&\\
\hat{J}^{2}=&J_{+}J_{-}+J_{z}^{2}-J_{z}\\
&\\
\end{array}
$$

We show here that these operators obey the commutation relations for angular momentum.

We can in turn express $\hat{H}$ in terms of the above quasispin operators and the number operator

$$
\hat{N}=\sum_{p\sigma}
a_{p\sigma}^{\dagger}a_{p\sigma}.
$$

We have the following quasispin operators

<!-- Equation labels as ordinary links -->
<div id="eq:Jpm"></div>

$$
\begin{equation}
J_{\pm} = \sum_p a_{p\pm}^\dagger a_{p\mp},
\label{eq:Jpm} \tag{1} 
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:Jz"></div>

$$
\begin{equation} 
J_{z} = \frac{1}{2}\sum_{p,\sigma} \sigma a_{p\sigma}^\dagger a_{p\sigma},
\label{eq:Jz} \tag{2} 
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:J2"></div>

$$
\begin{equation} 
J^{2} = J_+ J_- + J_z^2 - J_z,
\label{eq:J2} \tag{3}
\end{equation}
$$

and we want to compute the commutators

$$
[J_z,J_\pm], \quad [J_+,J_-], \quad [J^2,J_\pm] \quad \text{og} \quad 
[J^2,J_z].
$$

Let us start with the first one and inserting for  $J_z$ and $J_\pm$ given by the equations ([2](#eq:Jz)) and ([1](#eq:Jpm)), respectively, we obtain

$$
\begin{align*}
[J_z,J_\pm] &= J_z J_\pm - J_\pm J_z \\
%
&= \left( \frac{1}{2}\sum_{p,\sigma} \sigma a_{p\sigma}^\dagger a_{p\sigma} \right)
\left( \sum_{p'} a_{p'\pm}^\dagger a_{p'\mp} \right) -
\left( \sum_{p'} a_{p'\pm}^\dagger a_{p'\mp} \right)
\left( \frac{1}{2}\sum_{p,\sigma} \sigma a_{p\sigma}^\dagger a_{p\sigma} \right) \\
&= \frac{1}{2} \sum_{p,p',\sigma} \sigma \left( a_{p\sigma}^\dagger a_{p\sigma} a_{p'\pm}^\dagger a_{p'\mp} - a_{p'\pm}^\dagger a_{p'\mp} a_{p\sigma}^\dagger a_{p\sigma} \right).
\end{align*}
$$

Using the commutation relations for the creation and annihilation operators

<!-- Equation labels as ordinary links -->
<div id="eq:al,ak"></div>

$$
\begin{equation}
\{ a_l,a_k \} = 0, \label{eq:al,ak} \tag{4} 
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:ald,akd"></div>

$$
\begin{equation} 
\{ a_l^\dagger , a_k^\dagger \} = 0, \label{eq:ald,akd} \tag{5} 
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:ald,ak"></div>

$$
\begin{equation} 
\{ a_l^\dagger , a_k \} = \delta_{lk}, \label{eq:ald,ak} \tag{6}
\end{equation}
$$

in order to move the operators in the right product to be in the same order as those in the lefthand product

$$
\begin{align*}
[J_z,J_\pm] &= \frac{1}{2} \sum_{p,p',\sigma} \sigma \left(
a_{p\sigma}^\dagger a_{p\sigma} a_{p'\pm}^\dagger a_{p'\mp} -
a_{p'\pm}^\dagger \left( \delta_{p' p} \delta_{\mp \sigma} - a_{p\sigma}^\dagger a_{p'\mp} \right) a_{p\sigma} \right) \\
&= \frac{1}{2} \sum_{p,p',\sigma} \sigma \left(
a_{p\sigma}^\dagger a_{p\sigma} a_{p'\pm}^\dagger a_{p'\mp} -
a_{p'\pm}^\dagger \delta_{p' p} \delta_{\mp \sigma} a_{p\sigma} +
a_{p'\pm}^\dagger a_{p\sigma}^\dagger a_{p'\mp} a_{p\sigma} \right), \\
\end{align*}
$$

which results in

$$
\begin{align*}
[J_z,J_\pm]
&= \frac{1}{2} \sum_{p,p',\sigma} \sigma \left(
a_{p\sigma}^\dagger a_{p\sigma} a_{p'\pm}^\dagger a_{p'\mp} -
a_{p'\pm}^\dagger \delta_{pp'} \delta_{\mp \sigma} a_{p\sigma} +
a_{p\sigma}^\dagger a_{p'\pm}^\dagger a_{p\sigma} a_{p'\mp} \right) \\
&= \frac{1}{2} \sum_{p,p',\sigma} \sigma \left(
a_{p\sigma}^\dagger a_{p\sigma} a_{p'\pm}^\dagger a_{p'\mp} -
a_{p'\pm}^\dagger \delta_{pp'} \delta_{\mp \sigma} a_{p\sigma} +
a_{p\sigma}^\dagger \left( \delta_{pp'} \delta_{\pm \sigma} - a_{p\sigma} a_{p'\pm}^\dagger \right) a_{p'\mp} \right) \\
&= \frac{1}{2} \sum_{p,p',\sigma} \sigma \left(
a_{p\sigma}^\dagger \delta_{pp'} \delta_{\pm \sigma} a_{p'\mp} -
a_{p'\pm}^\dagger \delta_{pp'} \delta_{\mp \sigma} a_{p\sigma} \right). \\
\end{align*}
$$

The last equality leads to

$$
\begin{align*}
[J_z,J_\pm] &= \frac{1}{2} \sum_p \left(
(\pm 1) a_{p\pm}^\dagger a_{p\mp} - (\mp 1)
a_{p\pm}^\dagger a_{p\mp} \right) =
\pm \frac{1}{2} \sum_p \left(
a_{p\pm}^\dagger a_{p\mp} + (\pm 1)
a_{p\pm}^\dagger a_{p\mp} \right) \\
&= \pm \sum_p a_{p\pm}^\dagger a_{p\mp} = \pm J_\pm,
\end{align*}
$$

where the last results follows from comparing with Eq. ([1](#eq:Jpm)).

We can then continue with the next commutation relation, using Eq. ([1](#eq:Jpm)),

$$
\begin{align*}
[J_+,J_-] &= J_+ J_- - J_- J_+ \\
&= \left( \sum_p a_{p'+}^\dagger a_{p-} \right)
\left( \sum_{p'} a_{p'-}^\dagger a_{p'+} \right) -
\left( \sum_{p'} a_{p'-}^\dagger a_{p'+} \right)
\left( \sum_p a_{p+}^\dagger a_{p-} \right) \\
&= \sum_{p,p'} \left(
a_{p'+}^\dagger a_{p-} a_{p'-}^\dagger a_{p'+} -
a_{p'-}^\dagger a_{p'+} a_{p+}^\dagger a_{p-} \right) \\
&= \sum_{p,p'} \left(
a_{p'+}^\dagger a_{p-} a_{p'-}^\dagger a_{p'+} -
a_{p'-}^\dagger \left( \delta_{++} \delta_{pp'} -
a_{p+}^\dagger a_{p'+} \right) a_{p-} \right) \\
&= \sum_{p,p'} \left(
a_{p'+}^\dagger a_{p-} a_{p'-}^\dagger a_{p'+} -
a_{p'-}^\dagger \delta_{pp'} a_{p-} +
a_{p'-}^\dagger a_{p+}^\dagger a_{p'+} a_{p-} \right) \\
&= \sum_{p,p'} \left(
a_{p'+}^\dagger a_{p-} a_{p'-}^\dagger a_{p'+} -
a_{p'-}^\dagger \delta_{pp'} a_{p-} +
a_{p+}^\dagger a_{p'-}^\dagger a_{p-} a_{p'+} \right) \\
&= \sum_{p,p'} \left(
a_{p'+}^\dagger a_{p-} a_{p'-}^\dagger a_{p'+} -
a_{p'-}^\dagger \delta_{pp'} a_{p-} +
a_{p+}^\dagger \left( \delta_{--} \delta_{pp'} -
a_{p-} a_{p'-}^\dagger \right) a_{p'+} \right) \\
&= \sum_{p,p'} \left(
a_{p+}^\dagger \delta_{pp'} a_{p'+} -
a_{p'-}^\dagger \delta_{pp'} a_{p-} \right), \\
\end{align*}
$$

which results in

$$
[J_+,J_-] = \sum_p \left(
a_{p+}^\dagger a_{p+} -
a_{p-}^\dagger a_{p-} \right) = 2J_z,
$$

It is straightforward to show that

$$
[J^2, J_\pm] = [J_+ J_- + J_z^2 - J_z, J_\pm] =
[J_+ J_-, J_\pm] + [J_z^2, J_\pm] - [J_z, J_\pm].
$$

Using the relations

<!-- Equation labels as ordinary links -->
<div id="eq:ab,c"></div>

$$
\begin{equation}
[AB,C] = A[B,C] + [A,C]B, \label{eq:ab,c} \tag{7} 
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:a,bc"></div>

$$
\begin{equation} 
[A,BC] = [A,B]C + B[A,C], \label{eq:a,bc} \tag{8}
\end{equation}
$$

we obtain

$$
[J^2, J_\pm] =
J_+ [J_-,J_\pm] + [J_+,J_\pm] J_- + J_z [J_z,J_\pm] + [J_z,J_\pm] J_z - [J_z,J_\pm].
$$

Finally, from the above it follows that

$$
\begin{align*}
[J^2, J_+] &= -2J_+ J_z + J_z [J_z,J_+] + [J_z,J_+] J_z - [J_z,J_+] \\
&= -2J_+ J_z + J_z J_+ + J_+ J_z - J_+ \\
&= -2J_+ J_z + J_+ + J_+ J_z + J_+ J_z - J_+ = 0,
\end{align*}
$$

and

$$
\begin{align*}
[J^2, J_-] &= 2J_z J_- + J_z [J_z,J_-] + [J_z,J_-] J_z - [J_z,J_-] \\
&= 2J_z J_- - J_z J_- - J_- J_z + J_- \\
&= J_z J_- - (J_z J_- + J_-) + J_- = 0.
\end{align*}
$$

Our last commutator is given by

$$
\begin{align*}
[J^2,J_z] &= [J_+ J_- + J_z^2 - J_z, J_z] \\
&= [J_+ J_-, J_z] + [J_z^2, J_z] - [J_z, J_z] \\
&= J_+ [J_-, J_z] + [J_+,J_z] J_- \\
&= J_+ J_- - J_+ J_- = 0
\end{align*}
$$

Summing up we have

<!-- Equation labels as ordinary links -->
<div id="eq:kJzJpm"></div>

$$
\begin{equation}
[J_z, J_\pm] = \pm J_\pm, \label{eq:kJzJpm} \tag{9} 
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:kJpJm"></div>

$$
\begin{equation} 
[J_+, J_-] = 2J_z, \label{eq:kJpJm} \tag{10} 
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:kJ2Jpm"></div>

$$
\begin{equation} 
[J^2, J_\pm] = 0, \label{eq:kJ2Jpm} \tag{11} 
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:kJ2Jz"></div>

$$
\begin{equation} 
[J^2,J_z] = 0, \label{eq:kJ2Jz} \tag{12}
\end{equation}
$$

which are the standard commutation relations for angular (or orbital) momentum $L_\pm$, $L_z$ og $L^2$.

## Rewriting the Hamiltonian

We wrote the above Hamiltonian as

$$
H = H_0 + H_1 +H_2,
$$

with

$$
H_0 = \frac{1}{2} \varepsilon \sum_{p\sigma}\sigma a_{p\sigma}^{\dagger}a_{p\sigma},
$$

and

$$
H_1 = \frac{1}{2} V \sum_{p,p',\sigma} a_{p\sigma}^\dagger a_{p'\sigma}^\dagger a_{p'-\sigma} a_{p-\sigma},
$$

and

$$
H_{2} = \frac{1}{2} W \sum_{p,p',\sigma}a_{p\sigma}^\dagger a_{p'-\sigma}^\dagger a_{p'\sigma}a_{p-\sigma}.
$$

We will now rewrite the Hamiltonian in terms of the above quasi-spin operators and the number operator

<!-- Equation labels as ordinary links -->
<div id="eq:N"></div>

$$
\begin{equation}
N = \sum_{p,\sigma} a_{p\sigma}^\dagger a_{p\sigma}.
\label{eq:N} \tag{13}
\end{equation}
$$

Going through each term of the Hamiltonian and using the expressions for the quasi-spin operators we obtain

<!-- Equation labels as ordinary links -->
<div id="eq:H0ny"></div>

$$
\begin{equation}
H_0 = \varepsilon J_z.
\label{eq:H0ny} \tag{14}
\end{equation}
$$

Moving over to $H_1$ and using the anti-commutation relations ([4](#eq:al,ak)) through ([6](#eq:ald,ak)) we obtain

$$
\begin{align*}
H_1 &= \frac{1}{2} V \sum_{p,p',\sigma}
a_{p\sigma}^\dagger a_{p'\sigma}^\dagger a_{p'-\sigma} a_{p-\sigma} \\
&= \frac{1}{2} V \sum_{p,p',\sigma}
-a_{p\sigma}^\dagger a_{p'\sigma}^\dagger a_{p-\sigma} a_{p'-\sigma} \\
&= \frac{1}{2} V \sum_{p,p',\sigma}
-a_{p\sigma}^\dagger \left( \delta_{pp'} \delta_{\sigma -\sigma} - a_{p-\sigma} a_{p'\sigma}^\dagger \right) a_{p'-\sigma} \\
&= \frac{1}{2} V \sum_{p,p',\sigma}
a_{p\sigma}^\dagger a_{p-\sigma} a_{p'\sigma}^\dagger a_{p'-\sigma} \\
\end{align*}
$$

Rewriting the sum  over $\sigma$ we arrive at

$$
\begin{align*}
H_1 &= \frac{1}{2} V\sum_{p,p'}
a_{p+}^\dagger a_{p-} a_{p'+}^\dagger a_{p'-} +
a_{p-}^\dagger a_{p+} a_{p'-}^\dagger a_{p'+} \\
&= \frac{1}{2} V \left[ \sum_p \left( a_{p+}^\dagger a_{p-} \right)
\sum_{p'} \left( a_{p'+}^\dagger a_{p'-} \right) +
\sum_p \left( a_{p-}^\dagger a_{p+} \right)
\sum_{p'} \left( a_{p'-}^\dagger a_{p'+} \right) \right] \\
&= \frac{1}{2} V \left[ J_+ J_+ + J_- J_- \right] = \frac{1}{2} V \left[ J_+^2 + J_-^2 \right] ,
\end{align*}
$$

which leads to

<!-- Equation labels as ordinary links -->
<div id="eq:H1ny"></div>

$$
\begin{equation}
H_1 = \frac{1}{2} V \left( J_+^2 + J_-^2 \right).
\label{eq:H1ny} \tag{15}
\end{equation}
$$

Finally, we rewrite the last term

$$
\begin{align*}
H_2 &= \frac{1}{2} W \sum_{p,p',\sigma}
a_{p\sigma}^\dagger a_{p'-\sigma}^\dagger a_{p'\sigma} a_{p-\sigma} \\
&= \frac{1}{2} W \sum_{p,p',\sigma}
-a_{p\sigma}^\dagger a_{p'-\sigma}^\dagger a_{p-\sigma} a_{p'\sigma} \\
&= \frac{1}{2} W \sum_{p,p',\sigma}
-a_{p\sigma}^\dagger \left( \delta_{pp'} \delta_{-\sigma -\sigma} -
a_{p-\sigma} a_{p'-\sigma}^\dagger \right) a_{p'\sigma} \\
&= \frac{1}{2} W \sum_{p,p',\sigma}
-a_{p\sigma}^\dagger \delta_{pp'} a_{p'\sigma} +
a_{p\sigma}^\dagger a_{p-\sigma} a_{p'-\sigma}^\dagger a_{p'\sigma} \\
&= \frac{1}{2} W \left( -\sum_{p,\sigma}
a_{p\sigma}^\dagger a_{p\sigma} +
\sum_{p,p',\sigma} a_{p\sigma}^\dagger a_{p-\sigma} a_{p'-\sigma}^\dagger a_{p'\sigma} \right) \\
\end{align*}
$$

Using the expression for the number operator we obtain

$$
\begin{align*}
\sum_{p,p',\sigma} a_{p\sigma}^\dagger a_{p-\sigma} a_{p'-\sigma}^\dagger a_{p'\sigma}
&= \sum_{p,p'} a_{p+}^\dagger a_{p-} a_{p'-}^\dagger a_{p'+} +
a_{p-}^\dagger a_{p+} a_{p'+}^\dagger a_{p'-} \\
&= \sum_p \left( a_{p+}^\dagger a_{p-} \right)
\sum_{p'} \left( a_{p'-}^\dagger a_{p'+} \right) +
\sum_p \left( a_{p-}^\dagger a_{p+} \right)
\sum_{p'} \left( a_{p'+}^\dagger a_{p'-} \right) \\
&= J_+ J_- + J_- J_+,
\end{align*}
$$

resulting in

<!-- Equation labels as ordinary links -->
<div id="eq:H2ny"></div>

$$
\begin{equation}
H_2 = \frac{1}{2} W \left( -N + J_+ J_- + J_- J_+ \right).
\label{eq:H2ny} \tag{16}
\end{equation}
$$

We have thus expressed the Hamiltonian in term of the quasi-spin operators.

## Commutation relations for the Hamiltonian
The above expressions can in turn be used to show that the Hamiltonian commutes with the various quasi-spin operators. This leads to quantum numbers which are conserved.
Let us first show that $[H,J^2]=0$, which means that $J$ is a so-called *good* quantum number.
We have

$$
\begin{align*}
[H,J^2] &= [H_0 + H_1 + H_2,J^2] \\
&= [H_0,J^2] + [H_1,J^2] + [H_2,J^2] \\
&= \varepsilon [J_z,J^2] + \frac{1}{2} V [J_+^2 + J_-^2,J^2] +
\frac{1}{2} W [-N + J_+ J_- + J_- J_+,J^2]. \\
\end{align*}
$$

We have previously shown that

$$
[H,J^2] = \frac{1}{2} V \left( [J_+^2,J^2] + [J_-^2,J^2] \right) +
\frac{1}{2} W \left( -[N,J^2] + [J_+ J_-,J^2] + [J_- J_+, J^2] \right)
$$

Using that $[J_\pm,J^2] = 0$, it follows that $[J_\pm^2,J^2] = 0$. We can then see that $[J_+ J_-,J^2] = 0$ and $[J_- J_+, J^2] = 0$ which leads to

$$
\begin{align*}
[H,J^2] &= -\frac{1}{2} W [N,J^2] \\
&= \frac{1}{2} W \left( -[N,J_+ J_-] - [N,J_z^2] + [N,J_z] \right) \\
&= \frac{1}{2} W \left( -[N,J_+]J_- - J_+[N,J_-] - [N,J_z]J_z - J_z[N,J_z] + [N,J_z] \right).
\end{align*}
$$

Combining with the number operator we have

$$
\begin{align*}
[N,J_\pm] &= N J_\pm - J_\pm N \\
&= \left( \sum_{p,\sigma} a_{p\sigma}^\dagger a_{p\sigma} \right)
\left( \sum_{p'} a_{p'\pm}^\dagger a_{p'\mp} \right) -
\left( \sum_{p'} a_{p'\pm}^\dagger a_{p'\mp} \right)
\left( \sum_{p,\sigma} a_{p\sigma}^\dagger a_{p\sigma} \right) \\
&= \sum_{p,p',\sigma}
a_{p\sigma}^\dagger a_{p\sigma} a_{p'\pm}^\dagger a_{p'\mp} -
a_{p'\pm}^\dagger a_{p'\mp} a_{p\sigma}^\dagger a_{p\sigma} \\
&= \sum_{p,p',\sigma}
a_{p\sigma}^\dagger a_{p\sigma} a_{p'\pm}^\dagger a_{p'\mp} -
a_{p'\pm}^\dagger \left( \delta_{\mp \sigma} \delta_{pp'} - a_{p\sigma}^\dagger a_{p'\mp} \right) a_{p\sigma} \\
&= \sum_{p,p',\sigma}
a_{p\sigma}^\dagger a_{p\sigma} a_{p'\pm}^\dagger a_{p'\mp} -
a_{p'\pm}^\dagger \delta_{\mp \sigma} \delta_{pp'} a_{p\sigma} +
a_{p'\pm}^\dagger a_{p\sigma}^\dagger a_{p'\mp} a_{p\sigma} \\
&= \sum_{p,p',\sigma}
a_{p\sigma}^\dagger a_{p\sigma} a_{p'\pm}^\dagger a_{p'\mp} +
a_{p\sigma}^\dagger a_{p'\pm}^\dagger a_{p\sigma} a_{p'\mp} -
\sum_{p} a_{p\pm}^\dagger  a_{p\mp} \\
&= \sum_{p,p',\sigma}
a_{p\sigma}^\dagger a_{p\sigma} a_{p'\pm}^\dagger a_{p'\mp} +
a_{p\sigma}^\dagger \left( \delta_{pp'} \delta_{\pm \sigma} -
a_{p\sigma} a_{p'\pm}^\dagger \right) a_{p'\mp} -
\sum_{p} a_{p\pm}^\dagger  a_{p\mp} \\
&= \sum_p a_{p\pm}^\dagger a_{p\mp} -
\sum_{p} a_{p\pm}^\dagger  a_{p\mp} = 0. \\
\end{align*}
$$

We obtain then

$$
\begin{align*}
[N,J_z] &= N J_z - J_z N \\
&= \left( \sum_{p,\sigma} a_{p\sigma}^\dagger a_{p\sigma} \right)
\left( \frac{1}{2}\sum_{p',\sigma} \sigma a_{p'\sigma}^\dagger a_{p'\sigma} \right) -
\left( \frac{1}{2}\sum_{p',\sigma} \sigma a_{p'\sigma}^\dagger a_{p'\sigma} \right)
\left( \sum_{p,\sigma} a_{p\sigma}^\dagger a_{p\sigma} \right) \\
&= \sum_{p,p',\sigma} 
\sigma a_{p\sigma}^\dagger a_{p\sigma} a_{p'\sigma}^\dagger a_{p'\sigma} -
\sigma a_{p'\sigma}^\dagger a_{p'\sigma} a_{p\sigma}^\dagger a_{p\sigma} = 0,
\end{align*}
$$

which leads to

<!-- Equation labels as ordinary links -->
<div id="eq:kHJ2"></div>

$$
\begin{equation}
[H,J^2] = 0,
\label{eq:kHJ2} \tag{17}
\end{equation}
$$

and $J$ is a good quantum number.

### Constructing the Hamiltonian matrix for $J=2$

We start with the state (unique) where all spins point down

<!-- Equation labels as ordinary links -->
<div id="eq:2,-2"></div>

$$
\begin{equation}
\vert 2,-2\rangle = a_{1-}^{\dagger} a_{2-}^{\dagger}
a_{3-}^{\dagger} a_{4-}^{\dagger} \vert 0\rangle
\label{eq:2,-2} \tag{18}
\end{equation}
$$

which is a state with  $J_z = -2$ and $J = 2$. (we label the states as $\vert J,J_z\rangle$). For $J = 2$ we have the spin projections $J_z = -2,-1,0,1,2$.
We can use the lowering and raising operators for spin in order to define the other states

<!-- Equation labels as ordinary links -->
<div id="eq:J+ket"></div>

$$
\begin{equation}
J_+ \vert J,J_z\rangle = \sqrt{J(J+1) - J_z(J_z + 1)} \vert J,J_z + 1\rangle,
\label{eq:J+ket} \tag{19} 
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:J-ket"></div>

$$
\begin{equation} 
J_- \vert J,J_z\rangle = \sqrt{J(J+1) - J_z(J_z - 1)} \vert J,J_z - 1\rangle.
\label{eq:J-ket} \tag{20}
\end{equation}
$$

We can then construct all other states with $J=2$ using the raising operator
$J_+$ on $\vert 2,-2\rangle$

$$
J_+ \vert 2,-2\rangle = \sqrt{2(2+1) - (-2)(-2+1)} \vert 2,-2+1\rangle =\sqrt{6 - 2} \vert 2,-1\rangle = 2\vert 2,-1\rangle,
$$

which gives

<!-- Equation labels as ordinary links -->
<div id="_auto1"></div>

$$
\begin{equation}
\vert 2,-1\rangle = \frac{1}{2} J_+ \vert 2,-2\rangle \notag 
\label{_auto1} \tag{21}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto2"></div>

$$
\begin{equation} 
= \frac{1}{2} \sum_p a_{p+}^\dagger a_{p-} a_{1-}^{\dagger} a_{2-}^{\dagger}
a_{3-}^{\dagger} a_{4-}^{\dagger} \vert 0\rangle \notag 
\label{_auto2} \tag{22}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:2,-1"></div>

$$
\begin{equation} 
= \frac{1}{2} \left(
a_{1+}^{\dagger} a_{2-}^{\dagger} a_{3-}^{\dagger} a_{4-}^{\dagger} +
a_{1-}^{\dagger} a_{2+}^{\dagger} a_{3-}^{\dagger} a_{4-}^{\dagger} +
a_{1-}^{\dagger} a_{2-}^{\dagger} a_{3+}^{\dagger} a_{4-}^{\dagger} +
a_{1-}^{\dagger} a_{2-}^{\dagger} a_{3-}^{\dagger} a_{4+}^{\dagger}
\right) \vert 0\rangle. \label{eq:2,-1} \tag{23}
\end{equation}
$$

We can construct all the other states in the same way. That is

$$
J_+ \vert 2,-1\rangle = \sqrt{2(2+1) - (-1)(-1+1)} \vert 2,-1+1\rangle = \sqrt{6} \vert 2,0\rangle,
$$

which results in

<!-- Equation labels as ordinary links -->
<div id="eq:2,0"></div>

$$
\begin{equation}
\begin{aligned}
\vert 2,0\rangle &= \frac{1}{\sqrt{6}} \left(
a_{1+}^{\dagger} a_{2+}^{\dagger} a_{3-}^{\dagger} a_{4-}^{\dagger} +
a_{1+}^{\dagger} a_{2-}^{\dagger} a_{3+}^{\dagger} a_{4-}^{\dagger} +
a_{1+}^{\dagger} a_{2-}^{\dagger} a_{3-}^{\dagger} a_{4+}^{\dagger} +
a_{1-}^{\dagger} a_{2+}^{\dagger} a_{3+}^{\dagger} a_{4-}^{\dagger} + \right. \\
&\quad\,\, \left.
a_{1-}^{\dagger} a_{2+}^{\dagger} a_{3-}^{\dagger} a_{4+}^{\dagger} +
a_{1-}^{\dagger} a_{2-}^{\dagger} a_{3+}^{\dagger} a_{4+}^{\dagger} \right) \vert 0\rangle
\end{aligned}
\label{eq:2,0} \tag{24}
\end{equation}
$$

The two remaining states are

<!-- Equation labels as ordinary links -->
<div id="eq:2,1"></div>

$$
\begin{equation}
\vert2,1\rangle  = \frac{1}{2} \left(
a_{1+}^{\dagger} a_{2+}^{\dagger} a_{3+}^{\dagger} a_{4-}^{\dagger} +
a_{1+}^{\dagger} a_{2+}^{\dagger} a_{3-}^{\dagger} a_{4+}^{\dagger} +
a_{1+}^{\dagger} a_{2-}^{\dagger} a_{3+}^{\dagger} a_{4+}^{\dagger} +
a_{1-}^{\dagger} a_{2+}^{\dagger} a_{3+}^{\dagger} a_{4+}^{\dagger}
 \right).
\label{eq:2,1} \tag{25}
\end{equation}
$$

and

<!-- Equation labels as ordinary links -->
<div id="eq:2,2"></div>

$$
\begin{equation}
\vert 2,2\rangle  = a_{1+}^{\dagger} a_{2+}^{\dagger} a_{3+}^{\dagger} a_{4+}^{\dagger} \vert 0\rangle.
\label{eq:2,2} \tag{26}
\end{equation}
$$

These five states can in turn be used as computational basis states in order to define the Hamiltonian matrix to be diagonalized. The Hamiltonian is hermitian and we obtain after all this labor of ours

<!-- Equation labels as ordinary links -->
<div id="eq:HJ=2"></div>

$$
\begin{equation}
H_{J = 2} =
\begin{pmatrix}
-2\varepsilon & 0 & \sqrt{6}V & 0 & 0 \\
0 & -\varepsilon + 3W & 0 & 3V & 0 \\
\sqrt{6}V & 0 & 4W & 0 & \sqrt{6}V \\
0 & 3V & 0 & \varepsilon + 3W & 0 \\
0 & 0 & \sqrt{6}V & 0 & 2\varepsilon
\end{pmatrix}
\label{eq:HJ=2} \tag{27}
\end{equation}
$$

Inserting values for the constants $\epsilon$, $V$ and $W$ we can in turn diagonalize the above matrix and obtain the relevant eigenvalues.

## Mean field solution, Hartree-Fock theory  and comparison with exact diagonalization

This will be added later.

## Variational Quantum Eigensolver

One initial algorithm to estimate the eigen-energies of a quantum
Hamiltonian was quantum phase estimation. In it, one
encodes the eigen-energies, one binary bit at a time (up to $n$ bits),
into the complex phases of the quantum states of the Hilbert space for
$n$ qubits. It does this by applying powers of controlled unitary
evolution operators to a quantum state that can be expanded in terms
of the Hamiltonian's eigenvectors of interest. The eigen-energies are
encoded into the complex phases in such a way that taking the inverse
quantum Fourier transformation of the states into which the
eigen-energies are encoded results in a measurement probability
distribution that has peaks around the bit strings that represent a
binary fraction which corresponds to the eigen-energies of the quantum
state acted upon by the controlled unitary operators. While quantum
phase estimation (QPE) is provably efficient, non-hybrid, and
non-variational, the number of qubits and length of circuits required
is too great for our NISQ era quantum computers. Thus, QPE is only
efficiently applicable to large, fault-tolerant quantum computers that
likely won't exist in the near, but the far future.

Therefore, a different algorithm for finding the eigen-energies of a
quantum Hamiltonian was put forth in 2014 called the variational
quantum eigensolver, commonly referred to as VQE. The
algorithm is hybrid, meaning that it requires the use of both a
quantum computer and a classical computer. It is also variational,
meaning that it relies, ultimately, on solving an optimization problem
by varying parameters and thus is not as deterministic as QPE. The
variational quantum eigensolver is based on the variational principle:
The expectation value of a Hamiltonian $H$ in a state
$|\psi(\theta)\rangle$ parameterized by a set of angles $\theta$, is
always greater than or equal to the minimum eigen-energy $E_0$. To see
this, let $|n\rangle$ be the eigenstates of $H$, that is

<!-- Equation labels as ordinary links -->
<div id="_auto3"></div>

$$
\begin{equation}
H|n\rangle=E_n|n\rangle.
\label{_auto3} \tag{28}
\end{equation}
$$

We can then expand our state $|\psi(\theta)\rangle$ in terms of said eigenstates

<!-- Equation labels as ordinary links -->
<div id="_auto4"></div>

$$
\begin{equation}
|\psi(\theta)\rangle=\sum_nc_n|n\rangle,
\label{_auto4} \tag{29}
\end{equation}
$$

and plug this into the expectation value to yield

$$
\langle\psi(\theta)|H|\psi(\theta)\rangle
=
\sum_{nm}c^*_mc_n\langle m|H|n \rangle
\nonumber
$$

$$
=
\sum_{nm}c^*_mc_nE_n\langle m|n \rangle
\nonumber
$$

$$
=
\sum_{nm}\delta_{nm}c^*_mc_nE_n
\nonumber
$$

$$
=
\sum_{n}|c_n|^2E_n
\nonumber
$$

$$
\geq
E_0\sum_{n}|c_n|^2
\nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="_auto5"></div>

$$
\begin{equation} 
=
E_0,
\label{_auto5} \tag{30}
\end{equation}
$$

which implies that we can minimize over the set of angles $\theta$ and arrive at the ground state energy $E_0$

<!-- Equation labels as ordinary links -->
<div id="_auto6"></div>

$$
\begin{equation}
\min_\theta \ \langle\psi(\theta)|H|\psi(\theta)\rangle=E_0.
\label{_auto6} \tag{31}
\end{equation}
$$

Using this fact, the VQE algorithm can be broken down into the following steps
1. Prepare the variational state $|\psi(\theta)\rangle$ on a quantum computer.

2. Measure this circuit in various bases and send these measurements to a classical computer

3. The classical computer post-processes the measurement data to compute the expectation value $\langle\psi(\theta)|H|\psi(\theta)\rangle$

4. The classical computer varies the parameters $\theta$ according to a classical minimization algorithm and sends them back to the quantum computer which runs step 1 again.

This loop continues until the classical optimization algorithm
terminates which results in a set of angles $\theta_{\text{min}}$ that
characterize the ground state $|\phi(\theta_{\text{min}})\rangle$ and
an estimate for the ground state energy
$\langle\psi(\theta_{\text{min}})|H|\psi(\theta_{\text{min}})\rangle$.

### Expectation values

To execute the second step of VQE, we need to understand how expectation values of operators can be estimated via quantum computers by post-processing measurements of quantum circuits in different basis. To rotate bases, one uses the basis rotator $B_\sigma$ which is defined for each Pauli gate $\sigma$ to be

<!-- Equation labels as ordinary links -->
<div id="_auto7"></div>

$$
\begin{equation}
B_{\sigma} =
\begin{cases}
H,  \text{if} \ \sigma = X 
\label{_auto7} \tag{32}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto8"></div>

$$
\begin{equation} 
HS^{\dagger},  \text{if} \ \sigma = Y 
\label{_auto8} \tag{33}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto9"></div>

$$
\begin{equation} 
I,  \text{if} \ \sigma = Z 
\label{_auto9} \tag{34}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto10"></div>

$$
\begin{equation} 
\end{cases}.
\label{_auto10} \tag{35}
\end{equation}
$$

Note the following identity of the basis rotator

<!-- Equation labels as ordinary links -->
<div id="_auto11"></div>

$$
\begin{equation}
B^\dagger_\sigma Z B_\sigma = \sigma
, \label{_auto11} \tag{36}
\end{equation}
$$

which follows from the fact that $HZH=X$ and $SXS^\dagger=Y$. With this, we see that the expectation value of an arbitrary
Pauli-gate $\sigma$ in the state $\ket{\psi}$ can be expressed as a linear combination of probabilities

$$
E_{\psi}(\sigma)
= \mel{\psi}{\sigma}{\psi} \nonumber
$$

$$
=\mel{\psi}{B_{\sigma}^{\dagger}ZB_{\sigma}}{\psi} \nonumber
$$

$$
=\mel{\phi}{Z}{\phi} \nonumber
$$

$$
=\mel{\phi}{\left(\sum_{x\in\{0,1\}}(-1)^x\ket{x}\bra{x}\right)}{\phi} \nonumber
$$

$$
=\sum_{x\in\{0,1\}}(-1)^x\abs{\bra{x}\ket{\phi}}^2\nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="_auto12"></div>

$$
\begin{equation} 
=\sum_{x\in\{0,1\}}(-1)^xP(\ket{\phi}\to\ket{x}),
\label{_auto12} \tag{37}
\end{equation}
$$

where $\ket{\phi}=\ket{B_\sigma\phi}$ and $P(\ket{\phi}\to\ket{x}$ is the probability that the state $\ket{\phi}$ collapses to the state $\ket{x}$ when measured. This can be extended to any arbitrary Pauli string: consider the string of Pauli operators $P=\bigotimes_{p\in Q}\sigma_p$ which acts non-trivially on the set of qubits $Q$ which is a subset of the total set of $n$ qubits in the system. Then

$$
E_{\psi}\left(P\right)
=\bra{\psi}\left(\bigotimes_{p\in Q}\sigma_p\right)\ket{\psi} \nonumber
$$

$$
=\bra{\psi}\left(\bigotimes_{p\in Q}\sigma_p\right)
\left(\bigotimes_{q\notin Q}I_q\right)\ket{\psi} \nonumber
$$

$$
=\bra{\psi}\left(\bigotimes_{p \in Q}B_{\sigma_p}^{\dagger}Z_pB_{\sigma_p}\right)
\left(\bigotimes_{q\notin Q}I_q\right)\ket{\psi} \nonumber
$$

$$
=
\bra{\psi}\left(\bigotimes_{p \in Q}B_{\sigma_p}^{\dagger}\right)
\left(\bigotimes_{p \in Q}Z_p\right)
\left(\bigotimes_{q\notin Q}I_q\right)
\left(\bigotimes_{p \in Q}B_{\sigma_p}\right)\ket{\psi} \nonumber
$$

$$
=
\bra{\phi}
\left(\bigotimes_{p \in Q}Z_p\right)
\left(\bigotimes_{q\notin Q}I_q\right)
\ket{\phi} \nonumber
$$

$$
=
\bra{\phi}
\left(\bigotimes_{p\in Q}\sum_{x_p\in\{0_p,1_p\}}(-1)^{x_p}\ket{x_p}\bra{x_p}\right)
\left(\bigotimes_{q\notin Q}\sum_{y_q\in\{0_q,1_q\}}\ket{y_q}\bra{y_q}\right)
\ket{\phi} 
\nonumber
$$

$$
=
\bra{\phi}
\left(\sum_{x\in\{0,1\}^n}(-1)^{\sum_{p\in Q}x_p}\ket{x}\bra{x}\right)
\ket{\phi} 
\nonumber
$$

$$
=
\sum_{x\in\{0,1\}^n}(-1)^{\sum_{p\in Q}x_p}\abs{\bra{x}\ket{\phi}}^2
\nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="_auto13"></div> <div id="_auto13"></div>

$$
\begin{equation} 
=
\sum_{x\in\{0,1\}^n}(-1)^{\sum_{p\in Q}x_p}P(\ket{\phi}\to\ket{x})
, \label{_auto13} \tag{39} \tag{38}
\end{equation}
where $\ket{\phi}=\ket{\bigotimes_{p\in Q}B_{\sigma_p}\psi}$. Finally, because the expectation value is linear
\begin{equation}
E_\psi\left(\sum_{m}\lambda_mP_m\right) = \sum_m\lambda_mE_\psi(P_m),
\label{_auto13} \tag{39} \tag{38}
\end{equation}
$$

one can estimate any observable that can be written as a linear combination of Pauli-string terms.

### Measurement

To estimate the probability $P(\ket{\phi}\to \ket{x})$ from the
previous section, one prepares the state $\ket{\phi}$ on a quantum
computer and measures it, and then repeats this process (prepare and
measure) several times. The probability $P(\ket{\phi}\to \ket{x})$ is
estimated to be the number of times that one measures the bit-string
$x$ divided by the total number of measurements that one makes; that

<!-- Equation labels as ordinary links -->
<div id="_auto14"></div>

$$
\begin{equation}
P(\ket{\phi}\to \ket{x}) \approx \sum_{m=1}^M\frac{x_m}{M},
\label{_auto14} \tag{40}
\end{equation}
$$

where

<!-- Equation labels as ordinary links -->
<div id="_auto15"></div>

$$
\begin{equation}
x_m
=
\begin{cases}
1  \text{if the result of measurement is $x$} 
\label{_auto15} \tag{41}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto16"></div>

$$
\begin{equation} 
0  \text{if the result of measurement is not $x$}.
\end{cases}
\label{_auto16} \tag{42}
\end{equation}
$$

By the law of large numbers the approximation approaches equality as
$M$ goes to infinity

<!-- Equation labels as ordinary links -->
<div id="_auto17"></div>

$$
\begin{equation}
P(\ket{\phi}\to \ket{x}) = \lim_{M\to\infty}\sum_{m=1}^M\frac{x_m}{M}.
\label{_auto17} \tag{43}
\end{equation}
$$

As we obviously do not have infinite time nor infinite quantum
computers (which could be run in parallel), we must truncate our
number of measurement $M$ to a finite, but sufficiently large
number. More precisely, for precision $\epsilon$, each expectation
estimation subroutine within VQE requires $\mathcal{O}(1/\epsilon^2)$
samples from circuits with depth $\mathcal{O}(1)$.

**Lecture next week we continue  the discussion of the VQE and the Lipkin model.**

1. Discussion of the project

2. VQE examples and discussion of results